In [2]:
import pandas as pd
df2 = pd.read_csv("Merged_S&P_TSX_Composite_Historical_Data.csv")
df = df2.copy()
df.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,1990-01-02,"4,004.37","3,962.18","4,004.37","3,962.18",NaN,0.87%
1,1990-01-03,"4,009.47","4,014.33","4,020.86","4,007.19",NaN,0.13%
2,1990-01-04,"4,000.91","4,011.22","4,016.22","3,996.68",NaN,-0.21%
3,1990-01-05,"3,990.37","3,997.67","4,007.58","3,989.25",NaN,-0.26%
4,1990-01-08,"3,971.07","3,981.21","3,981.21","3,963.04",NaN,-0.48%


In [3]:
df = df.drop(columns=['Vol.'])

In [4]:
df.describe().T

,count,unique,top,freq
Date,6712,6712,1990-01-02,1
Price,6712,6689,"3,448.51",2
Open,6712,6687,"13,361.71",2
High,6712,6654,"4,525.67",2
Low,6712,6637,"11,447.20",3
Change %,6712,617,0.08%,53


In [5]:
df.isnull().sum()

Date        0
Price       0
Open        0
High        0
Low         0
Change %    0
dtype: int64

In [ ]:
# "Hac." sütununu dönüştürmek için fonksiyon
def convert_volume(value):
    value = value.replace(',', '.')  # Virgülü noktaya çevir
    if value.endswith('M'):
        return float(value[:-1]) * 1_000_000  # "M" ise milyonla çarp
    elif value.endswith('K'):
        return float(value[:-1]) * 1_000      # "K" ise binle çarp
    else:
        return float(value)                    # Kısaltma yoksa direkt float

# Fonksiyonu "Hac." sütununa uygulama
df['Hac.'] = df['Hac.'].apply(convert_volume)

In [6]:
df.head()

,Date,Price,Open,High,Low,Change %
0,1990-01-02,"4,004.37","3,962.18","4,004.37","3,962.18",0.87%
1,1990-01-03,"4,009.47","4,014.33","4,020.86","4,007.19",0.13%
2,1990-01-04,"4,000.91","4,011.22","4,016.22","3,996.68",-0.21%
3,1990-01-05,"3,990.37","3,997.67","4,007.58","3,989.25",-0.26%
4,1990-01-08,"3,971.07","3,981.21","3,981.21","3,963.04",-0.48%


In [8]:
# "Tarih" sütununu datetime formatına çevirme
#df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
df['Date'] = pd.to_datetime(df['Date'],format='%Y-%m-%d')
df.head()

,Date,Price,Open,High,Low,Change %
0,1990-01-02,"4,004.37","3,962.18","4,004.37","3,962.18",0.87%
1,1990-01-03,"4,009.47","4,014.33","4,020.86","4,007.19",0.13%
2,1990-01-04,"4,000.91","4,011.22","4,016.22","3,996.68",-0.21%
3,1990-01-05,"3,990.37","3,997.67","4,007.58","3,989.25",-0.26%
4,1990-01-08,"3,971.07","3,981.21","3,981.21","3,963.04",-0.48%


In [9]:
# Eksik verileri kontrol etme
print(df.isnull().sum())

# Eksik verileri bir önceki değerle doldurma
#df = df.fillna(method='ffill')

Date        0
Price       0
Open        0
High        0
Low         0
Change %    0
dtype: int64


In [10]:
# İlk birkaç satırı görüntüleme
print(df.head())

# Sütun tiplerini kontrol etme
print(df.dtypes)

        Date     Price      Open      High       Low Change %
0 1990-01-02  4,004.37  3,962.18  4,004.37  3,962.18    0.87%
1 1990-01-03  4,009.47  4,014.33  4,020.86  4,007.19    0.13%
2 1990-01-04  4,000.91  4,011.22  4,016.22  3,996.68   -0.21%
3 1990-01-05  3,990.37  3,997.67  4,007.58  3,989.25   -0.26%
4 1990-01-08  3,971.07  3,981.21  3,981.21  3,963.04   -0.48%
Date        datetime64[ns]
Price               object
Open                object
High                object
Low                 object
Change %            object
dtype: object


In [11]:
# "Fark" sütununu sayısal değere dönüştürme
df['Change %'] = df['Change %'].str.replace('%', '')      # % sembolünü kaldır
df['Change %'] = df['Change %'].str.replace(',', '.')     # Virgülü noktaya çevir
df['Change %'] = df['Change %'].astype(float)             # Float'a dönüştür
# Dönüşüm sonrası kontrol
print(df['Change %'].head())

0    0.87
1    0.13
2   -0.21
3   -0.26
4   -0.48
Name: Change %, dtype: float64


In [12]:
# Tarih sütununun zaten datetime64[ns] formatında olduğunu varsayıyoruz
# Fiyat sütunlarını sayısal formata dönüştürme
#for col in ['Price', 'Open', 'High', 'Low']:
    #df[col] = df[col].str.replace(',', '.').astype(float)

for col in ['Price', 'Open', 'High', 'Low']:
    # Binlik ayırıcı noktaları kaldır (örneğin "1.001.30" → "1001.30" değil, bu veride anlam bozulabilir ama yine deneyelim)
    df[col] = df[col].str.replace('.', '', regex=False)  # Önce tüm noktaları kaldır
    # Daha sonra ondalık ayırıcının da noktadan farklı olduğunu varsayarsak değiştirebilirsin
    df[col] = df[col].str.replace(',', '.', regex=False)  # Virgülü noktaya çevir
    df[col] = df[col].astype(float)

# Dönüşüm sonrası sütun tiplerini kontrol etme
print(df.dtypes)

Date        datetime64[ns]
Price              float64
Open               float64
High               float64
Low                float64
Change %           float64
dtype: object


In [13]:
#SMA (Simple Moving Average - Basit Hareketli Ortalama)
#EMA (Exponential Moving Average - Üssel Hareketli Ortalama)

# 10 günlük SMA ekleme
df['SMA_10'] = df['Price'].rolling(window=10).mean()

# 10 günlük EMA ekleme
df['EMA_10'] = df['Price'].ewm(span=10, adjust=False).mean()
df.head()

,Date,Price,Open,High,Low,Change %,SMA_10,EMA_10
0,1990-01-02,4.00437,3.96218,4.00437,3.96218,0.87,NaN,4.004370
1,1990-01-03,4.00947,4.01433,4.02086,4.00719,0.13,NaN,4.005297
2,1990-01-04,4.00091,4.01122,4.01622,3.99668,-0.21,NaN,4.004500
3,1990-01-05,3.99037,3.99767,4.00758,3.98925,-0.26,NaN,4.001931
4,1990-01-08,3.97107,3.98121,3.98121,3.96304,-0.48,NaN,3.996320


In [14]:
# 10 günlük SMA ekleme
df['SMA_10'] = df['Price'].rolling(window=10).mean()

# 10 günlük EMA ekleme
df['EMA_10'] = df['Price'].ewm(span=10, adjust=False).mean()
df.head()

,Date,Price,Open,High,Low,Change %,SMA_10,EMA_10
0,1990-01-02,4.00437,3.96218,4.00437,3.96218,0.87,NaN,4.004370
1,1990-01-03,4.00947,4.01433,4.02086,4.00719,0.13,NaN,4.005297
2,1990-01-04,4.00091,4.01122,4.01622,3.99668,-0.21,NaN,4.004500
3,1990-01-05,3.99037,3.99767,4.00758,3.98925,-0.26,NaN,4.001931
4,1990-01-08,3.97107,3.98121,3.98121,3.96304,-0.48,NaN,3.996320


In [15]:
df['SMA_10'] = df['SMA_10'].fillna(df['Price'].expanding().mean()) # ilk 10 gün ortalaması
df.head(20)

,Date,Price,Open,High,Low,Change %,SMA_10,EMA_10
0,1990-01-02,4.00437,3.96218,4.00437,3.96218,0.87,4.004370,4.004370
1,1990-01-03,4.00947,4.01433,4.02086,4.00719,0.13,4.006920,4.005297
2,1990-01-04,4.00091,4.01122,4.01622,3.99668,-0.21,4.004917,4.004500
3,1990-01-05,3.99037,3.99767,4.00758,3.98925,-0.26,4.001280,4.001931
4,1990-01-08,3.97107,3.98121,3.98121,3.96304,-0.48,3.995238,3.996320
5,1990-01-09,3.95235,3.97414,3.97414,3.95114,-0.47,3.988090,3.988325
6,1990-01-10,3.94156,3.94367,3.94606,3.93269,-0.27,3.981443,3.979822
7,1990-01-11,3.94170,3.94501,3.94749,3.93766,0.00,3.976475,3.972891
8,1990-01-12,3.88184,3.90761,3.90761,3.87969,-1.52,3.965960,3.956336
9,1990-01-15,3.89401,3.88575,3.89893,3.87299,0.31,3.958765,3.945004


In [16]:
from ta.momentum import RSIIndicator

#RSI (Relative Strength Index - Göreceli Güç Endeksi)
# 14 günlük RSI ekleme
rsi = RSIIndicator(close=df['Price'], window=14)
df['RSI_14'] = rsi.rsi()

In [17]:
#df['RSI_14'] = df['RSI_14'].fillna(df['RSI_14'].mean())

# İlk 30 günün RSI_14 ortalamasını hesapla (NaN hariç)
mean_first_20 = df.loc[:20, 'RSI_14'].mean()

# İlk 30 günün NaN değerlerini bu ortalama ile doldur
df.loc[:29, 'RSI_14'] = df.loc[:29, 'RSI_14'].fillna(mean_first_20)
df.head(20)

,Date,Price,Open,High,Low,Change %,SMA_10,EMA_10,RSI_14
0,1990-01-02,4.00437,3.96218,4.00437,3.96218,0.87,4.004370,4.004370,12.823872
1,1990-01-03,4.00947,4.01433,4.02086,4.00719,0.13,4.006920,4.005297,12.823872
2,1990-01-04,4.00091,4.01122,4.01622,3.99668,-0.21,4.004917,4.004500,12.823872
3,1990-01-05,3.99037,3.99767,4.00758,3.98925,-0.26,4.001280,4.001931,12.823872
4,1990-01-08,3.97107,3.98121,3.98121,3.96304,-0.48,3.995238,3.996320,12.823872
5,1990-01-09,3.95235,3.97414,3.97414,3.95114,-0.47,3.988090,3.988325,12.823872
6,1990-01-10,3.94156,3.94367,3.94606,3.93269,-0.27,3.981443,3.979822,12.823872
7,1990-01-11,3.94170,3.94501,3.94749,3.93766,0.00,3.976475,3.972891,12.823872
8,1990-01-12,3.88184,3.90761,3.90761,3.87969,-1.52,3.965960,3.956336,12.823872
9,1990-01-15,3.89401,3.88575,3.89893,3.87299,0.31,3.958765,3.945004,12.823872


In [18]:
df.dtypes

Date        datetime64[ns]
Price              float64
Open               float64
High               float64
Low                float64
Change %           float64
SMA_10             float64
EMA_10             float64
RSI_14             float64
dtype: object

In [19]:
from ta.trend import MACD

#MACD (Moving Average Convergence Divergence) Hareketli Ortalama Yakınsama Iraksama göstergesidir
# MACD ekleme (12, 26, 9)
macd = MACD(close=df['Price'], window_slow=26, window_fast=12, window_sign=9)
df['MACD'] = macd.macd()          # MACD çizgisi
df['MACD_Signal'] = macd.macd_signal()  # Sinyal çizgisi
df['MACD_Diff'] = macd.macd_diff()      # Fark (histogram)

In [20]:
df.head(40)

,Date,Price,Open,High,Low,Change %,SMA_10,EMA_10,RSI_14,MACD,MACD_Signal,MACD_Diff
0,1990-01-02,4.00437,3.96218,4.00437,3.96218,0.87,4.004370,4.004370,12.823872,NaN,NaN,NaN
1,1990-01-03,4.00947,4.01433,4.02086,4.00719,0.13,4.006920,4.005297,12.823872,NaN,NaN,NaN
2,1990-01-04,4.00091,4.01122,4.01622,3.99668,-0.21,4.004917,4.004500,12.823872,NaN,NaN,NaN
3,1990-01-05,3.99037,3.99767,4.00758,3.98925,-0.26,4.001280,4.001931,12.823872,NaN,NaN,NaN
4,1990-01-08,3.97107,3.98121,3.98121,3.96304,-0.48,3.995238,3.996320,12.823872,NaN,NaN,NaN
5,1990-01-09,3.95235,3.97414,3.97414,3.95114,-0.47,3.988090,3.988325,12.823872,NaN,NaN,NaN
6,1990-01-10,3.94156,3.94367,3.94606,3.93269,-0.27,3.981443,3.979822,12.823872,NaN,NaN,NaN
7,1990-01-11,3.94170,3.94501,3.94749,3.93766,0.00,3.976475,3.972891,12.823872,NaN,NaN,NaN
8,1990-01-12,3.88184,3.90761,3.90761,3.87969,-1.52,3.965960,3.956336,12.823872,NaN,NaN,NaN
9,1990-01-15,3.89401,3.88575,3.89893,3.87299,0.31,3.958765,3.945004,12.823872,NaN,NaN,NaN


In [21]:
for col in ['MACD', 'MACD_Signal', 'MACD_Diff']: # İlk günki değer ile dolduruldu
    first_value = df[col].iloc[33]
    df[col] = df[col].fillna(first_value)
df.isnull().sum()

Date           0
Price          0
Open           0
High           0
Low            0
Change %       0
SMA_10         0
EMA_10         0
RSI_14         0
MACD           0
MACD_Signal    0
MACD_Diff      0
dtype: int64

In [22]:
df.dtypes

Date           datetime64[ns]
Price                 float64
Open                  float64
High                  float64
Low                   float64
Change %              float64
SMA_10                float64
EMA_10                float64
RSI_14                float64
MACD                  float64
MACD_Signal           float64
MACD_Diff             float64
dtype: object

In [23]:
df3 = df.copy() # her ihtimale karşı verileri koruma
df3.head(40)

,Date,Price,Open,High,Low,Change %,SMA_10,EMA_10,RSI_14,MACD,MACD_Signal,MACD_Diff
0,1990-01-02,4.00437,3.96218,4.00437,3.96218,0.87,4.004370,4.004370,12.823872,-0.037495,-0.043813,0.006318
1,1990-01-03,4.00947,4.01433,4.02086,4.00719,0.13,4.006920,4.005297,12.823872,-0.037495,-0.043813,0.006318
2,1990-01-04,4.00091,4.01122,4.01622,3.99668,-0.21,4.004917,4.004500,12.823872,-0.037495,-0.043813,0.006318
3,1990-01-05,3.99037,3.99767,4.00758,3.98925,-0.26,4.001280,4.001931,12.823872,-0.037495,-0.043813,0.006318
4,1990-01-08,3.97107,3.98121,3.98121,3.96304,-0.48,3.995238,3.996320,12.823872,-0.037495,-0.043813,0.006318
5,1990-01-09,3.95235,3.97414,3.97414,3.95114,-0.47,3.988090,3.988325,12.823872,-0.037495,-0.043813,0.006318
6,1990-01-10,3.94156,3.94367,3.94606,3.93269,-0.27,3.981443,3.979822,12.823872,-0.037495,-0.043813,0.006318
7,1990-01-11,3.94170,3.94501,3.94749,3.93766,0.00,3.976475,3.972891,12.823872,-0.037495,-0.043813,0.006318
8,1990-01-12,3.88184,3.90761,3.90761,3.87969,-1.52,3.965960,3.956336,12.823872,-0.037495,-0.043813,0.006318
9,1990-01-15,3.89401,3.88575,3.89893,3.87299,0.31,3.958765,3.945004,12.823872,-0.037495,-0.043813,0.006318


In [24]:
df3.to_csv('Merged_S&P_TSX_without_Normalization.csv',index=False)

In [25]:
from sklearn.preprocessing import StandardScaler

# Normalleştirilecek sütunlar 'Change' ve 'Date' sütunu hariç her sütun normalize edilecek
normalize_columns = ['Price', 'Open', 'High', 'Low', 'SMA_10', 'EMA_10', 'MACD', 'MACD_Signal', 'MACD_Diff']

# StandardScaler ile standartlaştırma
scaler = StandardScaler()
df3[normalize_columns] = scaler.fit_transform(df3[normalize_columns])

# Sonucu kontrol etme
print(df3.head())

        Date     Price      Open      High       Low  Change %    SMA_10  \
0 1990-01-02 -1.270891 -1.281783 -1.274683 -1.277322      0.87 -1.270046   
1 1990-01-03 -1.269542 -1.267996 -1.270349 -1.265338      0.13 -1.269371   
2 1990-01-04 -1.271806 -1.268818 -1.271569 -1.268136     -0.21 -1.269902   
3 1990-01-05 -1.274594 -1.272401 -1.273840 -1.270115     -0.26 -1.270864   
4 1990-01-08 -1.279698 -1.276752 -1.280770 -1.277093     -0.48 -1.272464   

     EMA_10     RSI_14     MACD  MACD_Signal  MACD_Diff  
0 -1.270357  12.823872 -0.45228    -0.538521     0.2028  
1 -1.270112  12.823872 -0.45228    -0.538521     0.2028  
2 -1.270323  12.823872 -0.45228    -0.538521     0.2028  
3 -1.271003  12.823872 -0.45228    -0.538521     0.2028  
4 -1.272489  12.823872 -0.45228    -0.538521     0.2028  


In [26]:
df3.to_csv("Merged_S&P_TSX_normalization.csv",index=False)